In [ ]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
#from sklearn.model_selection import train_test_split

In [ ]:
item = torch.load('C://Users//user//Desktop//王茂田//twitch_item.pt')
duration=torch.load('C://Users//user//Desktop//王茂田//twitch_duration.pt')
interval=torch.load('C://Users//user//Desktop//王茂田//twitch_interval.pt')

In [ ]:
item = item.numpy().tolist()
duration = duration.numpy().tolist()
interval = interval.numpy().tolist()

In [ ]:
target_item=[]
target_duration=[]
target_interval=[]

for i in range (len(item)):
    target_item=target_item+(item[i][-1:])
    
for i in range (len(duration)):
    target_duration=target_duration+(duration[i][-1:])
    
for i in range (len(interval)):
    target_interval=target_interval+(interval[i][-1:])

In [ ]:
input_item=[]
input_duration=[]
input_interval=[]

for i in range (len(item)):
    input_item.append(item[i][:-1])
    
for i in range (len(duration)):
    input_duration.append(duration[i][:-1])
    
for i in range (len(interval)):
    input_interval.append(interval[i][:-1])

In [ ]:
lists_of_lists1=(input_item,target_item,input_duration,target_duration,input_interval,target_interval)

In [ ]:
import datetime
import math
import numpy as np
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F

In [ ]:
import argparse
import pickle
import time
from utils1 import build_graph, Data, split_validation
from model_twitch import *


parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='sample', help='dataset name: diginetica/yoochoose1_4/yoochoose1_64/sample')
parser.add_argument('--batchSize', type=int, default=8, help='input batch size')
parser.add_argument('--hiddenSize', type=int, default=256, help='hidden state size')
parser.add_argument('--epoch', type=int, default=30, help='the number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')  # [0.001, 0.0005, 0.0001]
parser.add_argument('--lr_dc', type=float, default=0.1, help='learning rate decay rate')
parser.add_argument('--lr_dc_step', type=int, default=3, help='the number of steps after which the learning rate decay')
parser.add_argument('--l2', type=float, default=1e-5, help='l2 penalty')  # [0.001, 0.0005, 0.0001, 0.00005, 0.00001]
parser.add_argument('--step', type=int, default=1, help='gnn propogation steps')
parser.add_argument('--patience', type=int, default=30, help='the number of epoch to wait before early stop ')
parser.add_argument('--nonhybrid', action='store_true', help='only use the global preference to predict')
parser.add_argument('--validation', action='store_true', help='validation')
parser.add_argument('--valid_portion', type=float, default=0.1, help='split the portion of training set as validation set')
opt = parser.parse_known_args()[0]

In [ ]:
import torch.optim as optim

# Hyperparameters
num_epochs = 30
batch_size = 8

num_games=16440
num_durations=95
num_intervals=21
game_embed_size=256
duration_embed_size=256
interval_embed_size=256
hidden_size=256
num_heads=1
num_layers=1

# Create model and optimizer
model = SequenceModel(opt,num_games, num_durations, num_intervals, game_embed_size, duration_embed_size, interval_embed_size, hidden_size, num_heads, num_layers)

In [ ]:
model

In [ ]:
import random
l1=random.sample(range(0,len(lists_of_lists1[0])),int(len(lists_of_lists1[0])*0.7))
train_data=[[],[],[],[],[],[]]
test_data=[[],[],[],[],[],[]]
for i in range(len(lists_of_lists1[0])):
    if i in set(l1):
        train_data[0].append(lists_of_lists1[0][i])
        train_data[1].append(lists_of_lists1[1][i])
        train_data[2].append(lists_of_lists1[2][i])
        train_data[3].append(lists_of_lists1[3][i])
        train_data[4].append(lists_of_lists1[4][i])
        train_data[5].append(lists_of_lists1[5][i])
    else:
        test_data[0].append(lists_of_lists1[0][i])
        test_data[1].append(lists_of_lists1[1][i])
        test_data[2].append(lists_of_lists1[2][i])
        test_data[3].append(lists_of_lists1[3][i])
        test_data[4].append(lists_of_lists1[4][i])
        test_data[5].append(lists_of_lists1[5][i])

In [ ]:


train_data1 = Data(train_data, shuffle=True)
test_data1 = Data(test_data, shuffle=False)
start = time.time()



bad_counter = 0
for epoch in range(opt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)
    hit_duration1,hit_interval1,hit_game3,mrr_game3,ndcg_game3,hit_duration2, mrr_duration2,ndcg_duration2,hit_interval2, mrr_interval2,ndcg_interval2,hit_game5,mrr_game5,ndcg_game5,hit_duration3, mrr_duration3,ndcg_duration3,hit_interval3, mrr_interval3,ndcg_interval3,hit_game10,mrr_game10,ndcg_game10,hit_duration5, mrr_duration5,ndcg_duration5,hit_interval5, mrr_interval5,ndcg_interval5= train_test(model,train_data1,test_data1)
    #hit, mrr, hid, A, seqh = train_test(model, train_data, test_data)
    flag = 0
    
    
  
        
    print('game預測:')
    print('\tHitRate@3:\t%.4f\tMMR@3:\t%.4f\tNDCG@3:\t%.4f\tEpoch:\t%d,'% (hit_game3, mrr_game3,ndcg_game3,epoch))
    print('\tHitRate@5:\t%.4f\tMMR@5:\t%.4f\tNDCG@5:\t%.4f\tEpoch:\t%d,'% (hit_game5, mrr_game5,ndcg_game5,epoch))
    print('\tHitRate@10:\t%.4f\tMMR@10:\t%.4f\tNDCG@10:\t%.4f\tEpoch:\t%d,'% (hit_game10, mrr_game10,ndcg_game10,epoch))
    
    print('Duration預測:')
    print('\tHitRate@1:\t%.4f\tEpoch:\t%d,'% (hit_duration1,epoch))
    print('\tHitRate@2:\t%.4f\tMMR@2:\t%.4f\tNDCG@2:\t%.4f\tEpoch:\t%d,'% (hit_duration2, mrr_duration2,ndcg_duration2,epoch))
    print('\tHitRate@3:\t%.4f\tMMR@3:\t%.4f\tNDCG@3:\t%.4f\tEpoch:\t%d,'% (hit_duration3, mrr_duration3,ndcg_duration3,epoch))
    print('\tHitRate@5:\t%.4f\tMMR@5:\t%.4f\tNDCG@5:\t%.4f\tEpoch:\t%d,'% (hit_duration5, mrr_duration5,ndcg_duration5,epoch))
    
    
    print('interval預測:')
    print('\tHitRate@1:\t%.4f\tEpoch:\t%d,'% (hit_interval1,epoch))
    print('\tHitRate@2:\t%.4f\tMMR@2:\t%.4f\tNDCG@2:\t%.4f\tEpoch:\t%d,'% (hit_interval2, mrr_interval2,ndcg_interval2,epoch))
    print('\tHitRate@3:\t%.4f\tMMR@3:\t%.4f\tNDCG@3:\t%.4f\tEpoch:\t%d,'% (hit_interval3, mrr_interval3,ndcg_interval3,epoch))
    print('\tHitRate@5:\t%.4f\tMMR@5:\t%.4f\tNDCG@5:\t%.4f\tEpoch:\t%d,'% (hit_interval5, mrr_interval5,ndcg_interval5,epoch))
    
    bad_counter += 1 - flag
    if bad_counter >= opt.patience:
        break

print('-------------------------------------------------------')
end = time.time()
print("Run time: %f s" % (end - start))